In [16]:
import pandas as pd

# New cell at index 0

path = r"../data/book_processed_output.csv"
df = pd.read_csv(path)

# Try common column names first, then try to auto-detect a 1-5 numeric column.
common = {"rating", "review_rating", "stars", "review_stars", "score"}
col = next((c for c in df.columns if c.lower() in common), None)

if col is None:
    for c in df.columns:
        s = pd.to_numeric(df[c], errors="coerce").dropna()
        if not s.empty and s.isin([1, 2, 3, 4, 5]).all():
            col = c
            break

if col is None:
    raise ValueError(f"Couldn't find a 1-5 rating column. Available columns: {list(df.columns)}")

counts = pd.to_numeric(df[col], errors="coerce").dropna().astype(int).value_counts().reindex(range(1, 6), fill_value=0).sort_index()
print(f"Rating counts for column '{col}':\n{counts}")

Rating counts for column 'rating':
rating
1      415
2     1201
3     3701
4     8104
5    13446
Name: count, dtype: int64


In [17]:
# Remap labels to 3-class sentiment: Negative (1,2) -> 0, Mixed (3) -> 1, Positive (4,5) -> 2
print("=" * 120)
print("REMAPPING LABELS TO 3-CLASS SENTIMENT")
print("=" * 120)

def remap_labels(y):
    """
    Remap 5-class ratings (1-5) to 3-class sentiment:
    1, 2 -> 0 (Negative)
    3 -> 1 (Mixed)
    4, 5 -> 2 (Positive)
    """
    return np.array([0 if rating in [1, 2] else (1 if rating == 3 else 2) for rating in y])

# Add remapped labels to dataframe
df['sentiment'] = df['rating'].apply(lambda x: 0 if x in [1, 2] else (1 if x == 3 else 2))

# Show mapping
print("\nLabel Remapping:")
print("  1, 2 stars -> 0 (Negative)")
print("  3 stars    -> 1 (Mixed)")
print("  4, 5 stars -> 2 (Positive)")

print("\nOverall Sentiment Distribution:")
sentiment_counts = df['sentiment'].value_counts().sort_index()
print(f"  Negative (0): {sentiment_counts.get(0, 0):,} reviews")
print(f"  Mixed (1):    {sentiment_counts.get(1, 0):,} reviews")
print(f"  Positive (2): {sentiment_counts.get(2, 0):,} reviews")

print("=" * 120)

REMAPPING LABELS TO 3-CLASS SENTIMENT

Label Remapping:
  1, 2 stars -> 0 (Negative)
  3 stars    -> 1 (Mixed)
  4, 5 stars -> 2 (Positive)

Overall Sentiment Distribution:
  Negative (0): 1,616 reviews
  Mixed (1):    3,701 reviews
  Positive (2): 22,180 reviews


In [21]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\kenpo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Required imports for 3-class sentiment classification
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name 'nn' from partially initialized module 'torch' (most likely due to a circular import) (C:\Users\kenpo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torch\__init__.py)

In [ ]:
# 3-Class Sentiment Classification (All Books Combined - 80/20 Train/Test)
print("=" * 120)
print("3-CLASS SENTIMENT CLASSIFICATION - COMBINED ALL BOOKS (80/20 Train/Test Split)")
print("=" * 120)

# Prepare data with sentiment labels (Negative=0, Mixed=1, Positive=2)
X_sentiment = df['lemmatized_text'].values # using lemmatized text
y_sentiment = df['sentiment'].astype(int).values

print(f"\n1. Data Summary:")
print(f"   Total reviews: {len(X_sentiment):,}")
print(f"   Sentiment distribution:")
print(f"     Negative (1-2★): {(y_sentiment == 0).sum():,} reviews")
print(f"     Mixed (3★):      {(y_sentiment == 1).sum():,} reviews")
print(f"     Positive (4-5★): {(y_sentiment == 2).sum():,} reviews")

# Split into 80/20 train/test with stratification
print(f"\n2. Splitting data (80/20 with stratification)...")
X_train_sent, X_test_sent, y_train_sent, y_test_sent = train_test_split(
    X_sentiment, y_sentiment, test_size=0.2, random_state=42, stratify=y_sentiment
)

print(f"   Training set: {len(X_train_sent):,} reviews")
print(f"   Test set: {len(X_test_sent):,} reviews")
print(f"   Training distribution:")
print(f"     Negative: {(y_train_sent == 0).sum():,} | Mixed: {(y_train_sent == 1).sum():,} | Positive: {(y_train_sent == 2).sum():,}")
print(f"   Test distribution:")
print(f"     Negative: {(y_test_sent == 0).sum():,} | Mixed: {(y_test_sent == 1).sum():,} | Positive: {(y_test_sent == 2).sum():,}")

# Load SBERT model
print(f"\n3. Loading Sentence-BERT model...")
model_sent = SentenceTransformer('all-MiniLM-L6-v2')
print(f"   ✓ Model loaded successfully (384-dimensional embeddings)")

# Generate embeddings for training set
print(f"\n4. Generating embeddings for training set...")
print(f"   Processing {len(X_train_sent):,} reviews...")
embeddings_train_sent = model_sent.encode(X_train_sent, show_progress_bar=True, batch_size=32)
print(f"   ✓ Shape: {embeddings_train_sent.shape}")

# Generate embeddings for test set
print(f"\n5. Generating embeddings for test set...")
print(f"   Processing {len(X_test_sent):,} reviews...")
embeddings_test_sent = model_sent.encode(X_test_sent, show_progress_bar=True, batch_size=32)
print(f"   ✓ Shape: {embeddings_test_sent.shape}")

# Train Random Forest classifier
print(f"\n6. Training Random Forest classifier on SBERT embeddings...")
classifier_sent = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
classifier_sent.fit(embeddings_train_sent, y_train_sent)
print(f"   ✓ Model trained successfully")

# Make predictions
print(f"\n7. Making predictions on test set...")
y_pred_sent = classifier_sent.predict(embeddings_test_sent)
accuracy_sent = accuracy_score(y_test_sent, y_pred_sent)
print(f"   ✓ Predictions completed")

# Display results
print(f"\n" + "=" * 120)
print(f"RESULTS - 3-CLASS SENTIMENT CLASSIFICATION (COMBINED)")
print(f"=" * 120)
print(f"\nAccuracy: {accuracy_sent:.4f} ({accuracy_sent*100:.2f}%)\n")

print(f"Classification Report:")
print(classification_report(y_test_sent, y_pred_sent, labels=[0, 1, 2], 
                          target_names=['Negative', 'Mixed', 'Positive'], digits=4))

# Confusion Matrix
cm_sent = confusion_matrix(y_test_sent, y_pred_sent, labels=[0, 1, 2])
print(f"Confusion Matrix:")
print(f"                Predicted Negative  Predicted Mixed  Predicted Positive")
print(f"Actual Negative     {cm_sent[0, 0]:>6}              {cm_sent[0, 1]:>6}             {cm_sent[0, 2]:>6}")
print(f"Actual Mixed        {cm_sent[1, 0]:>6}              {cm_sent[1, 1]:>6}             {cm_sent[1, 2]:>6}")
print(f"Actual Positive     {cm_sent[2, 0]:>6}              {cm_sent[2, 1]:>6}             {cm_sent[2, 2]:>6}")

print("\n" + "=" * 120)

3-CLASS SENTIMENT CLASSIFICATION - COMBINED ALL BOOKS (80/20 Train/Test Split)

1. Data Summary:
   Total reviews: 27,497
   Sentiment distribution:
     Negative (1-2★): 1,616 reviews
     Mixed (3★):      3,701 reviews
     Positive (4-5★): 22,180 reviews

2. Splitting data (80/20 with stratification)...
   Training set: 21,997 reviews
   Test set: 5,500 reviews
   Training distribution:
     Negative: 1,293 | Mixed: 2,961 | Positive: 17,743
   Test distribution:
     Negative: 323 | Mixed: 740 | Positive: 4,437

3. Loading Sentence-BERT model...
   ✓ Model loaded successfully (384-dimensional embeddings)

4. Generating embeddings for training set...
   Processing 21,997 reviews...


Batches: 100%|██████████| 688/688 [02:40<00:00,  4.30it/s]


   ✓ Shape: (21997, 384)

5. Generating embeddings for test set...
   Processing 5,500 reviews...


Batches: 100%|██████████| 172/172 [00:44<00:00,  3.89it/s]


   ✓ Shape: (5500, 384)

6. Training Random Forest classifier on SBERT embeddings...
   ✓ Model trained successfully

7. Making predictions on test set...
   ✓ Predictions completed

RESULTS - 3-CLASS SENTIMENT CLASSIFICATION (COMBINED)

Accuracy: 0.8085 (80.85%)

Classification Report:
              precision    recall  f1-score   support

    Negative     1.0000    0.0124    0.0245       323
       Mixed     0.6111    0.0149    0.0290       740
    Positive     0.8091    0.9989    0.8940      4437

    accuracy                         0.8085      5500
   macro avg     0.8067    0.3420    0.3158      5500
weighted avg     0.7936    0.8085    0.7266      5500

Confusion Matrix:
                Predicted Negative  Predicted Mixed  Predicted Positive
Actual Negative          4                   2                317
Actual Mixed             0                  11                729
Actual Positive          0                   5               4432



In [ ]:
# 3-Class Sentiment Classification by Individual Book (80/20 Train/Test)
print("=" * 120)
print("3-CLASS SENTIMENT CLASSIFICATION - BY INDIVIDUAL BOOK (80/20 Train/Test)")
print("=" * 120)

book_results_3class = {}

for idx, (book_id, book_df) in enumerate(df.groupby('book_id'), 1):
    print(f"\n[Book {idx}/8] Processing book_id: {book_id}")
    print(f"   Total reviews: {len(book_df):,}")
    
    # Get reviews and sentiment labels
    X_book = book_df['lemmatized_text'].values
    y_book = book_df['sentiment'].astype(int).values
    
    # Show sentiment distribution
    neg_count = (y_book == 0).sum()
    mix_count = (y_book == 1).sum()
    pos_count = (y_book == 2).sum()
    print(f"   Sentiment: Negative={neg_count:,} | Mixed={mix_count:,} | Positive={pos_count:,}")
    
    # Split data (80/20 with stratification)
    try:
        X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
            X_book, y_book, test_size=0.2, random_state=42, stratify=y_book
        )
        print(f"   Train: {len(X_train_b):,} | Test: {len(X_test_b):,}")
    except Exception as e:
        print(f"   ⚠️ Skipping (split error): {e}")
        continue
    
    # Generate embeddings
    try:
        emb_train_b = model_sent.encode(X_train_b, show_progress_bar=False, batch_size=32)
        emb_test_b = model_sent.encode(X_test_b, show_progress_bar=False, batch_size=32)
    except Exception as e:
        print(f"   ⚠️ Skipping (embedding error): {e}")
        continue
    
    # Train classifier
    try:
        clf_b = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
        clf_b.fit(emb_train_b, y_train_b)
        y_pred_b = clf_b.predict(emb_test_b)
        accuracy_b = accuracy_score(y_test_b, y_pred_b)
        
        print(f"   ✓ Accuracy: {accuracy_b:.4f} ({accuracy_b*100:.2f}%)")
        
        # Store results
        book_results_3class[book_id] = {
            'n_reviews': len(X_book),
            'n_train': len(X_train_b),
            'n_test': len(X_test_b),
            'accuracy': accuracy_b,
            'classifier': clf_b,
            'y_test': y_test_b,
            'y_pred': y_pred_b,
            'embeddings_train': emb_train_b,
            'embeddings_test': emb_test_b
        }
    except Exception as e:
        print(f"   ⚠️ Skipping (training error): {e}")
        continue

print("\n" + "=" * 120)
print("SUMMARY - 3-CLASS SENTIMENT BY INDIVIDUAL BOOK")
print("=" * 120)

if book_results_3class:
    summary_book_df = pd.DataFrame([
        {
            'Book ID': book_id,
            'Total Reviews': result['n_reviews'],
            'Train Set': result['n_train'],
            'Test Set': result['n_test'],
            'Accuracy': f"{result['accuracy']:.4f} ({result['accuracy']*100:.2f}%)"
        }
        for book_id, result in book_results_3class.items()
    ])
    print(summary_book_df.to_string(index=False))
    
    avg_accuracy_3c = np.mean([r['accuracy'] for r in book_results_3class.values()])
    print(f"\nAverage Accuracy: {avg_accuracy_3c:.4f} ({avg_accuracy_3c*100:.2f}%)")
else:
    print("No results available.")

print("=" * 120)

3-CLASS SENTIMENT CLASSIFICATION - BY INDIVIDUAL BOOK (80/20 Train/Test)

[Book 1/8] Processing book_id: 3636
   Total reviews: 5,365
   Sentiment: Negative=344 | Mixed=852 | Positive=4,169
   Train: 4,292 | Test: 1,073
   ✓ Accuracy: 0.7791 (77.91%)

[Book 2/8] Processing book_id: 136251
   Total reviews: 5,496
   Sentiment: Negative=102 | Mixed=237 | Positive=5,157
   Train: 4,396 | Test: 1,100
   ✓ Accuracy: 0.9382 (93.82%)

[Book 3/8] Processing book_id: 6218281
   Total reviews: 1,764
   Sentiment: Negative=156 | Mixed=397 | Positive=1,211
   Train: 1,411 | Test: 353
   ✓ Accuracy: 0.7025 (70.25%)

[Book 4/8] Processing book_id: 9938498
   Total reviews: 1,370
   Sentiment: Negative=139 | Mixed=358 | Positive=873
   Train: 1,096 | Test: 274
   ✓ Accuracy: 0.6387 (63.87%)

[Book 5/8] Processing book_id: 15507958
   Total reviews: 7,902
   Sentiment: Negative=359 | Mixed=810 | Positive=6,733
   Train: 6,321 | Test: 1,581
   ✓ Accuracy: 0.8526 (85.26%)

[Book 6/8] Processing book_id: